In [18]:
import requests
import json
import pandas as pd

from collections import Counter
import numpy as np
import plotly.express as px

In [2]:
def api_func(year):
    api_URL= f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/migr_asydcfsta?format=JSON&time={year}&unit=PER&citizen=LI&citizen=UK_OCT&citizen=BA&citizen=ME&citizen=MD&citizen=MK&citizen=GE&citizen=AL&citizen=RS&citizen=TR&citizen=UA&citizen=XK&citizen=AD&citizen=BY&citizen=VA&citizen=MC&citizen=RU&citizen=SM&citizen=AO&citizen=CM&citizen=CF&citizen=TD&citizen=CG&citizen=CD&citizen=GQ&citizen=GA&citizen=ST&citizen=BI&citizen=KM&citizen=DJ&citizen=ER&citizen=ET&citizen=KE&citizen=MG&citizen=MW&citizen=MU&citizen=MZ&citizen=RW&citizen=SC&citizen=SO&citizen=UG&citizen=TZ&citizen=ZM&citizen=ZW&citizen=DZ&citizen=EG&citizen=LY&citizen=MA&citizen=SS&citizen=SD&citizen=TN&citizen=EH&citizen=BW&citizen=LS&citizen=NA&citizen=ZA&citizen=SZ&citizen=BJ&citizen=BF&citizen=CV&citizen=CI&citizen=GM&citizen=GH&citizen=GN&citizen=GW&citizen=LR&citizen=ML&citizen=MR&citizen=NE&citizen=NG&citizen=SN&citizen=SL&citizen=TG&citizen=AG&citizen=BS&citizen=BB&citizen=CU&citizen=DM&citizen=DO&citizen=GD&citizen=HT&citizen=JM&citizen=KN&citizen=LC&citizen=VC&citizen=TT&citizen=BZ&citizen=CR&citizen=SV&citizen=GT&citizen=HN&citizen=MX&citizen=NI&citizen=PA&citizen=AR&citizen=BO&citizen=BR&citizen=CL&citizen=CO&citizen=EC&citizen=GY&citizen=PY&citizen=PE&citizen=SR&citizen=UY&citizen=VE&citizen=KZ&citizen=KG&citizen=TJ&citizen=TM&citizen=UZ&citizen=CN&citizen=JP&citizen=MN&citizen=KP&citizen=KR&citizen=TW&citizen=AF&citizen=BD&citizen=BT&citizen=IN&citizen=IR&citizen=MV&citizen=NP&citizen=PK&citizen=LK&citizen=BN&citizen=KH&citizen=ID&citizen=LA&citizen=MY&citizen=MM&citizen=PH&citizen=SG&citizen=TH&citizen=TL&citizen=VN&citizen=AM&citizen=AZ&citizen=BH&citizen=IQ&citizen=IL&citizen=JO&citizen=KW&citizen=LB&citizen=PS&citizen=OM&citizen=QA&citizen=SA&citizen=SY&citizen=AE&citizen=YE&citizen=AU&citizen=NZ&citizen=FJ&citizen=PG&citizen=SB&citizen=VU&citizen=KI&citizen=MH&citizen=FM&citizen=NR&citizen=PW&citizen=CK&citizen=WS&citizen=TO&citizen=TV&citizen=RNC&citizen=STLS&citizen=TOTAL&citizen=UNK&sex=T&sex=M&sex=F&sex=UNK&age=TOTAL&age=Y_LT14&age=Y14-17&age=Y_LT18&age=Y18-34&age=Y35-64&age=Y_GE65&age=UNK&decision=TOTAL&decision=TOTAL_POS&decision=REJECTED&lang=en"
    response = requests.get(api_URL)
    data = response.json()

    dimensions = data['dimension']
    values = data['value']


    rows = []


    dim_names = list(dimensions.keys())


    for index, value in values.items():
        row = {}
        

        idx = int(index)
        

        temp_idx = idx
        for dim_name in reversed(dim_names):
            dim_size = len(dimensions[dim_name]['category']['index'])
            dim_idx = temp_idx % dim_size
            temp_idx = temp_idx // dim_size
            
    
            dim_keys = list(dimensions[dim_name]['category']['index'].keys())
            category_key = dim_keys[dim_idx]
            
        
            if 'label' in dimensions[dim_name]['category']:
                row[dim_name] = dimensions[dim_name]['category']['label'].get(category_key, category_key)
            else:
                row[dim_name] = category_key
        
    
        row['value'] = value
        rows.append(row)


    df = pd.DataFrame(rows)
    asylum_drop = df.drop(["unit","freq"], axis =1 )
    asylum_drop = asylum_drop[asylum_drop["value"] != 0]
    asylum_drop = asylum_drop[(asylum_drop["age"] != "Total") & (asylum_drop["sex"] != "Total")]
    return asylum_drop


In [26]:
y2023_df = api_func("2023")
y2024_df = api_func("2024")
y2022_df = api_func("2022")
y2021_df = api_func("2021")

In [35]:
combined_df = pd.concat([y2021_df, y2022_df, y2023_df, y2024_df],ignore_index=True)
combined_df.head()

,time,geo,decision,age,sex,citizen,value
0,2021,Belgium,Rejected,From 14 to 17 years,Females,Afghanistan,5
1,2021,Germany,Rejected,From 14 to 17 years,Females,Afghanistan,40
2,2021,Greece,Rejected,From 14 to 17 years,Females,Afghanistan,70
3,2021,European Union - 27 countries (from 2020),Rejected,From 14 to 17 years,Females,Afghanistan,170
4,2021,France,Rejected,From 14 to 17 years,Females,Afghanistan,15


In [36]:
grouping = combined_df.groupby(["geo","citizen"])

In [37]:
grouping_age_decision =  combined_df.groupby(["age","decision"])
grouping_age_decision.describe()

value                           \
                                               count        mean          std   
age                 decision                                                    
65 years or over    Rejected                  1294.0   29.173107   107.171377   
                    Total                     1893.0   48.074485   215.686140   
                    Total positive decisions   992.0   52.419355   182.748783   
From 14 to 17 years Rejected                  2753.0   68.087541   365.649016   
                    Total                     3918.0  142.589331   954.331713   
                    Total positive decisions  2666.0  138.674044   817.824182   
From 18 to 34 years Rejected                  7304.0  403.578176  3870.911482   
                    Total                     8726.0  607.406028  6345.927597   
                    Total positive decisions  5270.0  445.926945  3917.677209   
From 35 to 64 years Rejected                  6533.0  212.146028  1774.850744   
                    Total                     7739.0  301.953095  2688.862696   
                    Total positive decisions  4369.0  217.127489  1527.057082   
Less than 14 years  Rejected                  4938.0  170.382746  1211.221248   
                    Total                     6530.0  346.825421  2866.487998   
                    Total positive decisions  4652.0  305.598667  2234.585557   
Less than 18 years  Rejected                  5334.0  193.210536  1417.943451   
                    Total                     7041.0  401.325806  3428.740461   
                    Total positive decisions  5058.0  354.527481  2688.209212   
Unknown             Rejected                    37.0   12.702703    11.761200   
                    Total                      105.0   16.952381    25.291524   
                    Total positive decisions    74.0   16.283784    19.880823   

                                                                                
                                              min  25%   50%     75%       max  
age                 decision                                                    
65 years or over    Rejected                  5.0  5.0   5.0   20.00    1495.0  
                    Total                     5.0  5.0  10.0   25.00    3950.0  
                    Total positive decisions  5.0  5.0  10.0   20.00    2475.0  
From 14 to 17 years Rejected                  5.0  5.0  10.0   30.00    8110.0  
                    Total                     5.0  5.0  15.0   45.00   27715.0  
                    Total positive decisions  5.0  5.0  10.0   40.00   19605.0  
From 18 to 34 years Rejected                  5.0  5.0  20.0   95.00  156745.0  
                    Total                     5.0  5.0  20.0  115.00  300945.0  
                    Total positive decisions  5.0  5.0  15.0   70.00  144200.0  
From 35 to 64 years Rejected                  5.0  5.0  15.0   65.00   64850.0  
                    Total                     5.0  5.0  20.0   80.00  109090.0  
                    Total positive decisions  5.0  5.0  15.0   50.00   44240.0  
Less than 14 years  Rejected                  5.0  5.0  15.0   55.00   30730.0  
                    Total                     5.0  5.0  20.0   75.00   77510.0  
                    Total positive decisions  5.0  5.0  15.0   55.00   48650.0  
Less than 18 years  Rejected                  5.0  5.0  15.0   63.75   38840.0  
                    Total                     5.0  5.0  20.0   85.00  105225.0  
                    Total positive decisions  5.0  5.0  15.0   65.00   66385.0  
Unknown             Rejected                  5.0  5.0  10.0   10.00      45.0  
                    Total                     5.0  5.0   5.0   15.00     145.0  
                    Total positive decisions  5.0  5.0   5.0   15.00     100.0

In [38]:
Counter(combined_df['sex'])

Counter({'Males': 46485, 'Females': 40429, 'Unknown': 312})

In [40]:
Counter(combined_df['age'])

Counter({'From 18 to 34 years': 21300,
         'From 35 to 64 years': 18641,
         'Less than 18 years': 17433,
         'Less than 14 years': 16120,
         'From 14 to 17 years': 9337,
         '65 years or over': 4179,
         'Unknown': 216})

In [41]:
Counter(combined_df['decision'])

Counter({'Total': 35952, 'Rejected': 28193, 'Total positive decisions': 23081})

In [42]:
Counter(combined_df['citizen'])

Counter({'Total': 4360,
         'Syria': 2772,
         'Afghanistan': 2718,
         'Iraq': 2170,
         'Russia': 2097,
         'Iran': 1933,
         'Somalia': 1919,
         'Türkiye': 1899,
         'Georgia': 1600,
         'Pakistan': 1575,
         'Nigeria': 1554,
         'Eritrea': 1520,
         'Ukraine': 1510,
         'Venezuela': 1457,
         'Democratic Republic of the Congo': 1292,
         'Cameroon': 1287,
         'Colombia': 1264,
         'Morocco': 1243,
         'Stateless': 1207,
         'Egypt': 1197,
         'Palestine*': 1195,
         'Albania': 1156,
         'Sudan': 1121,
         'Ethiopia': 1113,
         'Yemen': 1097,
         'Belarus': 1084,
         'Armenia': 1054,
         'Algeria': 1049,
         'Guinea': 1039,
         'China': 1010,
         'Lebanon': 979,
         'Unknown': 951,
         'India': 903,
         'Bangladesh': 897,
         'Tunisia': 878,
         'Sri Lanka': 876,
         'El Salvador': 864,
         'Azerbaij

In [43]:
Counter(combined_df['geo'])

Counter({'European Union - 27 countries (from 2020)': 14241,
         'France': 10153,
         'Germany': 8180,
         'Italy': 5848,
         'Belgium': 5660,
         'Sweden': 5441,
         'Spain': 5305,
         'Netherlands': 3896,
         'Greece': 3761,
         'Switzerland': 3441,
         'Austria': 3249,
         'Ireland': 2249,
         'Cyprus': 2002,
         'Finland': 1569,
         'Poland': 1349,
         'Romania': 1311,
         'Denmark': 1151,
         'Norway': 1129,
         'Luxembourg': 944,
         'Czechia': 862,
         'Iceland': 793,
         'Bulgaria': 745,
         'Lithuania': 664,
         'Malta': 649,
         'Portugal': 541,
         'Latvia': 460,
         'Croatia': 341,
         'Slovenia': 315,
         'Montenegro': 313,
         'Estonia': 308,
         'Slovakia': 231,
         'Hungary': 107,
         'Liechtenstein': 18})